In [160]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer



In [161]:
df = pd.read_csv('winequalityN.csv')
set(df[df.columns[-1]])
set(df["type"])

{'red', 'white'}

In [162]:
# Perform one-hot encoding
onehot_encoder = OneHotEncoder(sparse=False, drop='if_binary')
onehot_encoded = onehot_encoder.fit_transform(df[['type']])
df['output'] = onehot_encoded
df.drop("type", axis=1, inplace=True)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [169]:
# Separate the features (X) and target variable (y)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
# Check for NaN values in y
nan_indices = pd.isnull(y)

In [170]:
# Handle NaN values (e.g., remove or impute)
y = y[~nan_indices]  # Remove NaN values from y
X = X[~nan_indices.flatten()]  # Remove corresponding samples from X

In [171]:
X.shape

(6497, 12)

In [172]:
set(y)

{0.0, 1.0}

In [173]:
preprocessor = ColumnTransformer(transformers=[('num', SimpleImputer(strategy='median'), slice(0, None))])

# Preprocess the numerical columns
X = preprocessor.fit_transform(X)

In [174]:
y

array([1., 1., 1., ..., 0., 0., 0.])

In [175]:
over = RandomOverSampler()
X,y = over.fit_resample(X,y)

In [176]:
data = np.concatenate((X, y.reshape(-1, 1)), axis=1)

In [177]:

data = np.hstack(
    (X,np.reshape(y,(-1,1)))
)

In [178]:
y

array([1., 1., 1., ..., 0., 0., 0.])

In [180]:
data = pd.DataFrame(data,columns=df.columns)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,output
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6.0,1.0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6.0,1.0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6.0,1.0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6.0,1.0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9791,12.2,0.34,0.50,2.4,0.066,10.0,21.0,1.00000,3.12,1.18,9.2,6.0,0.0
9792,7.3,0.98,0.05,2.1,0.061,20.0,49.0,0.99705,3.31,0.55,9.7,3.0,0.0
9793,10.6,0.42,0.48,2.7,0.065,5.0,18.0,0.99720,3.21,0.87,11.3,6.0,0.0
9794,12.8,0.84,0.63,2.4,0.088,13.0,35.0,0.99970,3.10,0.60,10.4,6.0,0.0


In [181]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [182]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,output
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6.0,1.0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6.0,1.0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6.0,1.0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6.0,1.0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6.0,1.0


In [184]:
(len(data[data["output"] == 0]),len(data[data["output"] == 1]))


(4898, 4898)

In [185]:
X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=.4,random_state=0)
X_valid,X_test,y_valid,y_test = train_test_split(X_temp,y_temp,test_size=.5,random_state=0)

In [186]:
model= tf.keras.Sequential([
    tf.keras.layers.Dense(16,activation="relu"),
     tf.keras.layers.Dense(16,activation="relu"),
     tf.keras.layers.Dense(1,activation="sigmoid")#0 or 1
])

In [187]:

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)


In [188]:
model.evaluate(X_train,y_train)


184/184 [==============================] - 1s 3ms/step - loss: 0.6558 - accuracy: 0.6971


[0.6558029651641846, 0.6971243619918823]

In [189]:
model.evaluate(X_valid,y_valid)


62/62 [==============================] - 1s 8ms/step - loss: 0.6439 - accuracy: 0.7131


[0.6438999772071838, 0.7131189107894897]

In [190]:
model.fit(
    X_train,y_train,batch_size=16,epochs=20,validation_data=(X_valid,y_valid)
)

Epoch 1/20
368/368 [==============================] - 6s 6ms/step - loss: 0.1860 - accuracy: 0.9508 - val_loss: 0.0505 - val_accuracy: 0.9913
Epoch 2/20
368/368 [==============================] - 2s 4ms/step - loss: 0.0486 - accuracy: 0.9901 - val_loss: 0.0383 - val_accuracy: 0.9944
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.0396 - accuracy: 0.9927 - val_loss: 0.0352 - val_accuracy: 0.9944
Epoch 4/20
368/368 [==============================] - 1s 2ms/step - loss: 0.0355 - accuracy: 0.9935 - val_loss: 0.0330 - val_accuracy: 0.9944
Epoch 5/20
368/368 [==============================] - 1s 2ms/step - loss: 0.0320 - accuracy: 0.9940 - val_loss: 0.0301 - val_accuracy: 0.9949
Epoch 6/20
368/368 [==============================] - 1s 2ms/step - loss: 0.0292 - accuracy: 0.9942 - val_loss: 0.0284 - val_accuracy: 0.9949
Epoch 7/20
368/368 [==============================] - 1s 3ms/step - loss: 0.0269 - accuracy: 0.9946 - val_loss: 0.0266 - val_accuracy: 0.9949
Epoch 

In [191]:
model.evaluate(X_test,y_test)

62/62 [==============================] - 0s 1ms/step - loss: 0.0335 - accuracy: 0.9929


[0.033519722521305084, 0.9928571581840515]